# Imports and Setup


In [ ]:
import torch, torchvision, os, warnings
import pandas as pd
import torchvision.models as models
import torch.nn as nn

from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from collections import Counter
from tqdm.auto import tqdm
from typing import Dict, List, Tuple
from pathlib import Path

warnings.filterwarnings("ignore")
device = "cuda" if torch.cuda.is_available() else "cpu"

print(f"Pytorch Version -- {torch.__version__}")
print(f"torchvision Version -- {torchvision.__version__}")

seed = 42
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)

BATCH_SIZE = 32
EPOCHS = 100
PATIENCE = 10
IMG_SIZE = (224, 224)
NUM_WORKERS = os.cpu_count()
RGB_MEAN = [0.485, 0.456, 0.406]
RGB_STD = [0.229, 0.224, 0.225]

TRAIN_DIR = r"/kaggle/input/aqua20/train"
TEST_DIR = r"/kaggle/input/aqua20/test"

TARGET_DIR = r"/kaggle/working/results"
if not os.path.exists(TARGET_DIR):
    os.makedirs(TARGET_DIR)

Pytorch Version -- 2.5.1+cu124
torchvision Version -- 0.20.1+cu124


# Utility Functions


In [ ]:
def create_dataloaders(
    train_dir: str,
    test_dir: str,
    train_transform: transforms.Compose,
    test_transform: transforms.Compose,
    batch_size: int,
    num_workers: int = NUM_WORKERS,
):
    # Use ImageFolder to create dataset(s)
    train_data = datasets.ImageFolder(train_dir, transform=train_transform)
    test_data = datasets.ImageFolder(test_dir, transform=test_transform)

    # Get class names
    class_names = train_data.classes

    # Turn images into data loaders
    train_dataloader = DataLoader(
        train_data,
        batch_size=batch_size,
        shuffle=True,
        num_workers=num_workers,
        pin_memory=True,
    )
    test_dataloader = DataLoader(
        test_data,
        batch_size=batch_size,
        shuffle=False,
        num_workers=num_workers,
        pin_memory=True,
    )

    return train_dataloader, test_dataloader, class_names


def count_samples_per_class(dataloader):
    class_count = Counter()
    for _, labels in dataloader:
        class_count.update(labels.tolist())
    return class_count


def save_model(model: torch.nn.Module, target_dir: str, model_name: str):
    # Create target directory
    target_dir_path = Path(target_dir)
    target_dir_path.mkdir(parents=True, exist_ok=True)

    # Create model save path
    assert model_name.endswith(".pth") or model_name.endswith(
        ".pt"
    ), "model_name should end with '.pt' or '.pth'"
    model_save_path = target_dir_path / model_name

    # Save the model state_dict()
    print(f"[INFO] Saving model to: {model_save_path}")
    torch.save(obj=model.state_dict(), f=model_save_path)


def load_model(model: torch.nn.Module, model_path: str) -> torch.nn.Module:
    # Check if the model path exists
    model_path = Path(model_path)
    assert model_path.exists(), f"Model path {model_path} does not exist."

    # Load the state_dict
    print(f"[INFO] Loading model from: {model_path}")
    state_dict = torch.load(f=model_path, weights_only=True)

    # Load the state_dict into the model
    model.load_state_dict(state_dict)

    return model


def train_step(
    model: torch.nn.Module,
    dataloader: torch.utils.data.DataLoader,
    loss_fn: torch.nn.Module,
    optimizer: torch.optim.Optimizer,
    device: torch.device,
) -> Tuple[float, float]:
    # Put model in train mode
    model.train()

    # Setup train loss and train accuracy values
    train_loss, train_acc = 0, 0

    # Loop through data loader data batches
    for batch, (X, y) in enumerate(dataloader):
        # Send data to target device
        X, y = X.to(device), y.to(device)

        # 1. Forward pass
        y_pred = model(X)

        # 2. Calculate and accumulate loss
        loss = loss_fn(y_pred, y)  # y_pred now has shape (BATCH_SIZE, 22)
        train_loss += loss.item()

        # 3. Optimizer zero grad
        optimizer.zero_grad()

        # 4. Loss backward
        loss.backward()

        # 5. Optimizer step
        optimizer.step()

        # Calculate and accumulate accuracy metric across all batches
        _, y_pred_class = torch.max(y_pred, 1)  # Get predicted class indices
        train_acc += (y_pred_class == y).sum().item() / len(y_pred)

        if torch.cuda.is_available():
            torch.cuda.empty_cache()

    # Adjust metrics to get average loss and accuracy per batch
    train_loss = train_loss / len(dataloader)
    train_acc = train_acc / len(dataloader)
    return train_loss, train_acc


def test_step(
    model: torch.nn.Module,
    dataloader: torch.utils.data.DataLoader,
    loss_fn: torch.nn.Module,
    device: torch.device,
) -> Tuple[float, float]:
    # Put model in eval mode
    model.eval()

    # Setup test loss and test accuracy values
    test_loss, test_acc = 0, 0

    # Turn on inference context manager
    with torch.inference_mode():
        # Loop through DataLoader batches
        for batch, (X, y) in enumerate(dataloader):
            # Send data to target device
            X, y = X.to(device), y.to(device)

            # 1. Forward pass
            test_pred_logits = model(X)

            # 2. Calculate and accumulate loss
            loss = loss_fn(test_pred_logits, y)
            test_loss += loss.item()

            # Calculate and accumulate accuracy
            _, test_pred_labels = torch.max(
                test_pred_logits, 1
            )  # Get predicted class indices
            test_acc += (test_pred_labels == y).sum().item() / len(test_pred_labels)

    # Adjust metrics to get average loss and accuracy per batch
    test_loss = test_loss / len(dataloader)
    test_acc = test_acc / len(dataloader)
    return test_loss, test_acc


def train(
    model: torch.nn.Module,
    train_dataloader: torch.utils.data.DataLoader,
    test_dataloader: torch.utils.data.DataLoader,
    optimizer: torch.optim.Optimizer,
    loss_fn: torch.nn.Module,
    epochs: int,
    device: torch.device,
    patience: int = 5,
) -> Dict[str, List]:
    # Make sure model on target device
    model.to(device)

    # Create empty results dictionary
    results = {"train_loss": [], "train_acc": [], "test_loss": [], "test_acc": []}

    best_loss = float("inf")  # Initialize best loss
    epochs_without_improvement = 0  # Counter for early stopping
    clf_only_epochs = 0

    # Loop through training and testing steps for a number of epochs
    print("\ntraining only CLASSIFIER\n")
    for epoch in tqdm(range(epochs)):
        train_loss, train_acc = train_step(
            model=model,
            dataloader=train_dataloader,
            loss_fn=loss_fn,
            optimizer=optimizer,
            device=device,
        )
        test_loss, test_acc = test_step(
            model=model, dataloader=test_dataloader, loss_fn=loss_fn, device=device
        )

        # Print out what's happening
        print(
            f"Epoch: {epoch+1} | "
            f"train_loss: {train_loss:.4f} | "
            f"train_acc: {train_acc:.4f} | "
            f"test_loss: {test_loss:.4f} | "
            f"test_acc: {test_acc:.4f}"
        )

        # Update results dictionary
        results["train_loss"].append(train_loss)
        results["train_acc"].append(train_acc)
        results["test_loss"].append(test_loss)
        results["test_acc"].append(test_acc)

        # Early stopping logic
        if test_loss < best_loss:  # If the test loss has improved
            best_loss = test_loss  # Update the best loss
            epochs_without_improvement = 0  # Reset the counter
            # Saving the Model
            save_model(
                model=model, target_dir=TARGET_DIR, model_name=model_name + "_best.pth"
            )
        else:
            epochs_without_improvement += 1  # Increment the counter

        if (
            epochs_without_improvement >= patience
        ):  # Check if patience has been exceeded
            clf_only_epochs = epoch + 1
            print(f"Early stopping triggered at {epoch+1} epochs")
            break  # Exit the training loop

    model = load_model(
        model=model, model_path=TARGET_DIR + "/" + model_name + "_best.pth"
    )

    # Loop through training and testing steps for a number of epochs
    print("\nFULL MODEL TRAINING\n")
    epochs_without_improvement = 0

    for param in model.parameters():
        param.requires_grad = True

    for epoch in tqdm(range(epochs - clf_only_epochs)):
        train_loss, train_acc = train_step(
            model=model,
            dataloader=train_dataloader,
            loss_fn=loss_fn,
            optimizer=optimizer,
            device=device,
        )

        test_loss, test_acc = test_step(
            model=model, dataloader=test_dataloader, loss_fn=loss_fn, device=device
        )

        # Print out what's happening
        print(
            f"Epoch: {epoch+clf_only_epochs+2} | "
            f"train_loss: {train_loss:.4f} | "
            f"train_acc: {train_acc:.4f} | "
            f"test_loss: {test_loss:.4f} | "
            f"test_acc: {test_acc:.4f}"
        )

        # Update results dictionary
        results["train_loss"].append(train_loss)
        results["train_acc"].append(train_acc)
        results["test_loss"].append(test_loss)
        results["test_acc"].append(test_acc)

        # Early stopping logic
        if test_loss < best_loss:  # If the test loss has improved
            best_loss = test_loss  # Update the best loss
            epochs_without_improvement = 0  # Reset the counter
            # Saving the Model
            save_model(
                model=model, target_dir=TARGET_DIR, model_name=model_name + "_best.pth"
            )
        else:
            epochs_without_improvement += 1  # Increment the counter

        if (
            epochs_without_improvement >= patience
        ):  # Check if patience has been exceeded
            print(f"Early stopping triggered at {epoch+1} epochs")
            break  # Exit the training loop

    model = load_model(
        model=model, model_path=TARGET_DIR + "/" + model_name + "_best.pth"
    )

    # Return the filled results at the end of the epochs
    return results

# Loading Data


In [3]:
train_transform = transforms.Compose(
    [
        transforms.Resize(IMG_SIZE),
        transforms.ToTensor(),
        transforms.Normalize(mean=RGB_MEAN, std=RGB_STD),
    ]
)


test_transform = transforms.Compose(
    [
        transforms.Resize(IMG_SIZE),
        transforms.ToTensor(),
        transforms.Normalize(mean=RGB_MEAN, std=RGB_STD),
    ]
)


train_dataloader, test_dataloader, class_names = create_dataloaders(
    train_dir=TRAIN_DIR,
    test_dir=TEST_DIR,
    train_transform=train_transform,
    test_transform=test_transform,
    batch_size=BATCH_SIZE,
    num_workers=NUM_WORKERS,
)


NUM_CLASSES = len(class_names)


# Get exact number of train and test images
num_train_images = len(train_dataloader.dataset)
num_test_images = len(test_dataloader.dataset)

print(f"Exact number of train images: {num_train_images}")
print(f"Exact number of test images: {num_test_images}")
print(f"Class names: {class_names}")

Exact number of train images: 6559
Exact number of test images: 1612
Class names: ['coral', 'crab', 'diver', 'eel', 'fish', 'fishInGroups', 'flatworm', 'jellyfish', 'marine_dolphin', 'octopus', 'rayfish', 'seaAnemone', 'seaCucumber', 'seaSlug', 'seaUrchin', 'shark', 'shrimp', 'squid', 'starfish', 'turtle']


In [4]:
# Count samples in train and test dataloaders
train_class_count = count_samples_per_class(train_dataloader)
test_class_count = count_samples_per_class(test_dataloader)

# Create DataFrame
df = pd.DataFrame(
    {
        "Class": class_names,
        "Train Samples": [train_class_count[i] for i in range(len(class_names))],
        "Test Samples": [test_class_count[i] for i in range(len(class_names))],
    }
)

# Add a column for total samples
df["Total Samples"] = df["Train Samples"] + df["Test Samples"]

# Add a "Total" row at the bottom
total_row = pd.DataFrame(
    {
        "Class": ["Total"],
        "Train Samples": [df["Train Samples"].sum()],
        "Test Samples": [df["Test Samples"].sum()],
        "Total Samples": [df["Total Samples"].sum()],
    }
)

# Concatenate the original DataFrame with the total row
df = pd.concat([df, total_row], ignore_index=True)

# Save to CSV
df.to_csv(TARGET_DIR + "/statistical_summary.csv", index=None)
print(df)

             Class  Train Samples  Test Samples  Total Samples
0            coral           1562           348           1910
1             crab             43            11             54
2            diver             51            13             64
3              eel            160            41            201
4             fish           2200           538           2738
5     fishInGroups            272            72            344
6         flatworm             50            13             63
7        jellyfish             97            25            122
8   marine_dolphin             20            10             30
9          octopus             20            10             30
10         rayfish            382            95            477
11      seaAnemone            890           221           1111
12     seaCucumber             35            10             45
13         seaSlug             79            20             99
14       seaUrchin            115            29        

# Models


In [5]:
mobilenet_v2 = models.mobilenet_v2(weights=models.MobileNet_V2_Weights.IMAGENET1K_V1)
mobilenet_v2.classifier = nn.Sequential(
    nn.Dropout(p=0.2, inplace=False),
    nn.Linear(in_features=1280, out_features=NUM_CLASSES, bias=True),
)

resnet50 = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V1)
resnet50.fc = nn.Linear(in_features=2048, out_features=NUM_CLASSES, bias=True)

densenet121 = models.densenet121(weights=models.DenseNet121_Weights.IMAGENET1K_V1)
densenet121.classifier = nn.Linear(
    in_features=1024, out_features=NUM_CLASSES, bias=True
)

efficientnet_b0 = models.efficientnet_b0(
    weights=models.EfficientNet_B0_Weights.IMAGENET1K_V1
)
efficientnet_b0.classifier = nn.Sequential(
    nn.Dropout(p=0.2, inplace=False),
    nn.Linear(in_features=1280, out_features=NUM_CLASSES, bias=True),
)

vit_b_16 = models.vit_b_16(weights=models.ViT_B_16_Weights.IMAGENET1K_V1)
vit_b_16.heads = nn.Linear(in_features=768, out_features=NUM_CLASSES, bias=True)

convnext = models.convnext_base(weights=models.ConvNeXt_Base_Weights.IMAGENET1K_V1)
convnext.classifier[2] = nn.Linear(
    in_features=1024, out_features=NUM_CLASSES, bias=True
)

maxvit = models.maxvit_t(weights=models.MaxVit_T_Weights.IMAGENET1K_V1)
maxvit.classifier = nn.Sequential(
    nn.AdaptiveAvgPool2d(output_size=1),
    nn.Flatten(start_dim=1, end_dim=-1),
    nn.LayerNorm((512,), eps=1e-05, elementwise_affine=True),
    nn.Linear(in_features=512, out_features=512, bias=True),
    nn.Tanh(),
    nn.Linear(in_features=512, out_features=NUM_CLASSES, bias=False),
)


shufflenet_v2_x1_0 = models.shufflenet_v2_x1_0(
    weights=models.ShuffleNet_V2_X1_0_Weights.IMAGENET1K_V1
)
shufflenet_v2_x1_0.fc = nn.Linear(in_features=1024, out_features=NUM_CLASSES, bias=True)

squeezenet1_1 = models.squeezenet1_1(weights=models.SqueezeNet1_1_Weights.IMAGENET1K_V1)
squeezenet1_1.classifier[1] = nn.Conv2d(
    512,
    NUM_CLASSES,
    kernel_size=(1, 1),
    stride=(1, 1),
)

vgg19 = models.vgg19(weights=models.VGG19_Weights.IMAGENET1K_V1)
vgg19.classifier[6] = nn.Linear(in_features=4096, out_features=NUM_CLASSES, bias=True)

swin_v2_b = models.swin_v2_b(weights=models.Swin_V2_B_Weights.IMAGENET1K_V1)
swin_v2_b.head = nn.Linear(in_features=1024, out_features=NUM_CLASSES, bias=True)

regnet_x_32gf = models.regnet_x_32gf(weights=models.RegNet_X_32GF_Weights.IMAGENET1K_V1)
regnet_x_32gf.fc = nn.Linear(in_features=2520, out_features=NUM_CLASSES, bias=True)

inception_v3 = models.inception_v3(weights=None, aux_logits=False)
pretrained_state_dict = models.Inception_V3_Weights.IMAGENET1K_V1.get_state_dict(
    progress=True
)
inception_v3.load_state_dict(pretrained_state_dict, strict=False)
inception_v3.fc = nn.Linear(2048, NUM_CLASSES)
inception_v3.avgpool = nn.AdaptiveAvgPool2d((1, 1))


model_dict = {
    "shufflenet_v2_x1_0": shufflenet_v2_x1_0,
    "squeezenet1_1": squeezenet1_1,
    "vgg19": vgg19,
    "swin_v2_b": swin_v2_b,
    "regnet_x_32gf": regnet_x_32gf,
    "inception_v3": inception_v3,
    "mobilenet_v2": mobilenet_v2,
    "resnet50": resnet50,
    "densenet121": densenet121,
    "efficientnet_b0": efficientnet_b0,
    "vit_b_16": vit_b_16,
    "convnext": convnext,
    "maxvit": maxvit,
}

print("Models Used\n")
print("\n".join([str(i) for i in model_dict.keys()]))

Downloading: "https://download.pytorch.org/models/mobilenet_v2-b0353104.pth" to /root/.cache/torch/hub/checkpoints/mobilenet_v2-b0353104.pth
100%|██████████| 13.6M/13.6M [00:00<00:00, 114MB/s]
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 218MB/s]
Downloading: "https://download.pytorch.org/models/densenet121-a639ec97.pth" to /root/.cache/torch/hub/checkpoints/densenet121-a639ec97.pth
100%|██████████| 30.8M/30.8M [00:00<00:00, 163MB/s]
Downloading: "https://download.pytorch.org/models/efficientnet_b0_rwightman-7f5810bc.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b0_rwightman-7f5810bc.pth
100%|██████████| 20.5M/20.5M [00:00<00:00, 167MB/s]
Downloading: "https://download.pytorch.org/models/vit_b_16-c867db91.pth" to /root/.cache/torch/hub/checkpoints/vit_b_16-c867db91.pth
100%|██████████| 330M/330M [00:01<00:00, 212MB/s]
Downloading: "https://downloa

Models Used

shufflenet_v2_x1_0
squeezenet1_1
vgg19
swin_v2_b
regnet_x_32gf
inception_v3
mobilenet_v2
resnet50
densenet121
efficientnet_b0
vit_b_16
convnext
maxvit


In [6]:
def freeze_all_but_classifier(model, classifier_layers):
    for param in model.parameters():
        param.requires_grad = False

    # Enable gradient only for the classifier layers
    for layer in classifier_layers:
        for param in layer.parameters():
            param.requires_grad = True


# Freeze layers accordingly for each model
freeze_all_but_classifier(mobilenet_v2, [mobilenet_v2.classifier])
freeze_all_but_classifier(resnet50, [resnet50.fc])
freeze_all_but_classifier(densenet121, [densenet121.classifier])
freeze_all_but_classifier(efficientnet_b0, [efficientnet_b0.classifier])
freeze_all_but_classifier(vit_b_16, [vit_b_16.heads])
freeze_all_but_classifier(convnext, [convnext.classifier[2]])
freeze_all_but_classifier(maxvit, [maxvit.classifier])
freeze_all_but_classifier(shufflenet_v2_x1_0, [shufflenet_v2_x1_0.fc])
freeze_all_but_classifier(squeezenet1_1, [squeezenet1_1.classifier])
freeze_all_but_classifier(vgg19, [vgg19.classifier])
freeze_all_but_classifier(swin_v2_b, [swin_v2_b.head])
freeze_all_but_classifier(regnet_x_32gf, [regnet_x_32gf.fc])
freeze_all_but_classifier(inception_v3, [inception_v3.fc])


print("All but classifier layers frozen.")

All but classifier layers frozen.


# Model Parameters


In [7]:
model_params = [
    (model_name, sum(p.numel() for p in model.parameters()))
    for model_name, model in model_dict.items()
]


df = pd.DataFrame(model_params, columns=["Model Name", "Number of Parameters"])


csv_file = TARGET_DIR + "/model_parameters.csv"


df.to_csv(csv_file, index=False)


print(f"CSV file '{csv_file}' saved successfully.")

CSV file '/kaggle/working/results/model_parameters.csv' saved successfully.


# Training


In [8]:
for model_name, model in model_dict.items():
    print(f"\n*** {model_name} ***\n")
    # Loss Function And Optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

    # Training Loop
    results = train(
        model=model,
        train_dataloader=train_dataloader,
        test_dataloader=test_dataloader,
        optimizer=optimizer,
        loss_fn=criterion,
        epochs=EPOCHS,
        device=device,
        patience=PATIENCE,
    )

    # Saving the results
    df = pd.DataFrame(results)
    df.index.name = "iteration"
    df.to_csv(
        TARGET_DIR
        + "/results_"
        + model_name
        + "_"
        + str(df.index.max() + 1)
        + "_epochs.csv"
    )

    # Saving the Model
    save_model(
        model=model,
        target_dir=TARGET_DIR,
        model_name=model_name + "_" + str(df.index.max() + 1) + "_epochs.pth",
    )

    if torch.cuda.is_available():
        torch.cuda.empty_cache()


*** shufflenet_v2_x1_0 ***


training only CLASSIFIER



  0%|          | 0/100 [00:00<?, ?it/s]

Epoch: 1 | train_loss: 2.1809 | train_acc: 0.3405 | test_loss: 1.9711 | test_acc: 0.3744
[INFO] Saving model to: /kaggle/working/results/shufflenet_v2_x1_0_best.pth
Epoch: 2 | train_loss: 1.7871 | train_acc: 0.4485 | test_loss: 1.7723 | test_acc: 0.5165
[INFO] Saving model to: /kaggle/working/results/shufflenet_v2_x1_0_best.pth
Epoch: 3 | train_loss: 1.6247 | train_acc: 0.5461 | test_loss: 1.6327 | test_acc: 0.5600
[INFO] Saving model to: /kaggle/working/results/shufflenet_v2_x1_0_best.pth
Epoch: 4 | train_loss: 1.4985 | train_acc: 0.5695 | test_loss: 1.5091 | test_acc: 0.5858
[INFO] Saving model to: /kaggle/working/results/shufflenet_v2_x1_0_best.pth
Epoch: 5 | train_loss: 1.3949 | train_acc: 0.5887 | test_loss: 1.4168 | test_acc: 0.6225
[INFO] Saving model to: /kaggle/working/results/shufflenet_v2_x1_0_best.pth
Epoch: 6 | train_loss: 1.3080 | train_acc: 0.6073 | test_loss: 1.3314 | test_acc: 0.6475
[INFO] Saving model to: /kaggle/working/results/shufflenet_v2_x1_0_best.pth
Epoch: 7 |

  0%|          | 0/100 [00:00<?, ?it/s]

Epoch: 2 | train_loss: 0.6612 | train_acc: 0.7884 | test_loss: 0.7035 | test_acc: 0.7700
Epoch: 3 | train_loss: 0.3305 | train_acc: 0.8875 | test_loss: 0.7074 | test_acc: 0.7686
Epoch: 4 | train_loss: 0.1990 | train_acc: 0.9337 | test_loss: 0.7602 | test_acc: 0.7835
Epoch: 5 | train_loss: 0.1675 | train_acc: 0.9440 | test_loss: 0.8759 | test_acc: 0.7663
Epoch: 6 | train_loss: 0.1594 | train_acc: 0.9460 | test_loss: 0.8057 | test_acc: 0.7937
Epoch: 7 | train_loss: 0.1407 | train_acc: 0.9504 | test_loss: 0.8262 | test_acc: 0.8039
Epoch: 8 | train_loss: 0.1257 | train_acc: 0.9581 | test_loss: 0.8758 | test_acc: 0.7755
Epoch: 9 | train_loss: 0.1160 | train_acc: 0.9642 | test_loss: 0.8553 | test_acc: 0.7935
Epoch: 10 | train_loss: 0.1010 | train_acc: 0.9669 | test_loss: 0.8351 | test_acc: 0.8019
Epoch: 11 | train_loss: 0.1144 | train_acc: 0.9620 | test_loss: 0.8565 | test_acc: 0.7876
Early stopping triggered at 10 epochs
[INFO] Loading model from: /kaggle/working/results/shufflenet_v2_x1_0_

...

Epoch: 38 | train_loss: 1.1185 | train_acc: 0.7027 | test_loss: 0.6818 | test_acc: 0.8015
Epoch: 39 | train_loss: 0.6485 | train_acc: 0.8045 | test_loss: 0.7305 | test_acc: 0.7823
Epoch: 40 | train_loss: 0.5569 | train_acc: 0.8285 | test_loss: 0.6297 | test_acc: 0.8141
Epoch: 41 | train_loss: 0.4270 | train_acc: 0.8649 | test_loss: 0.8121 | test_acc: 0.7657
Epoch: 42 | train_loss: 0.3917 | train_acc: 0.8751 | test_loss: 0.8141 | test_acc: 0.7904
Epoch: 43 | train_loss: 0.3038 | train_acc: 0.9052 | test_loss: 0.7526 | test_acc: 0.7839
Epoch: 44 | train_loss: 0.2171 | train_acc: 0.9332 | test_loss: 0.9625 | test_acc: 0.7541
Epoch: 45 | train_loss: 0.2579 | train_acc: 0.9172 | test_loss: 0.8776 | test_acc: 0.7725
Epoch: 46 | train_loss: 0.2308 | train_acc: 0.9263 | test_loss: 1.0947 | test_acc: 0.7369
Epoch: 47 | train_loss: 0.1738 | train_acc: 0.9469 | test_loss: 1.1152 | test_acc: 0.7486
Early stopping triggered at 10 epochs
[INFO] Loading model from: /kaggle/working/results/convnext_be

  0%|          | 0/100 [00:00<?, ?it/s]

Epoch: 1 | train_loss: 0.6052 | train_acc: 0.8157 | test_loss: 0.4687 | test_acc: 0.8456
[INFO] Saving model to: /kaggle/working/results/maxvit_best.pth
Epoch: 2 | train_loss: 0.4147 | train_acc: 0.8646 | test_loss: 0.4782 | test_acc: 0.8597
Epoch: 3 | train_loss: 0.3621 | train_acc: 0.8805 | test_loss: 0.4778 | test_acc: 0.8556
Epoch: 4 | train_loss: 0.3190 | train_acc: 0.8921 | test_loss: 0.4735 | test_acc: 0.8550
Epoch: 5 | train_loss: 0.2997 | train_acc: 0.8931 | test_loss: 0.4553 | test_acc: 0.8636
[INFO] Saving model to: /kaggle/working/results/maxvit_best.pth
Epoch: 6 | train_loss: 0.2729 | train_acc: 0.9062 | test_loss: 0.4877 | test_acc: 0.8482
Epoch: 7 | train_loss: 0.2559 | train_acc: 0.9103 | test_loss: 0.4586 | test_acc: 0.8580
Epoch: 8 | train_loss: 0.2430 | train_acc: 0.9148 | test_loss: 0.4448 | test_acc: 0.8640
[INFO] Saving model to: /kaggle/working/results/maxvit_best.pth
Epoch: 9 | train_loss: 0.2224 | train_acc: 0.9213 | test_loss: 0.4617 | test_acc: 0.8634
Epoch: 

  0%|          | 0/82 [00:00<?, ?it/s]

Epoch: 20 | train_loss: 0.7636 | train_acc: 0.7798 | test_loss: 0.6874 | test_acc: 0.8031
Epoch: 21 | train_loss: 0.4799 | train_acc: 0.8431 | test_loss: 0.6026 | test_acc: 0.8297
Epoch: 22 | train_loss: 0.3951 | train_acc: 0.8765 | test_loss: 0.6960 | test_acc: 0.8056
Epoch: 23 | train_loss: 0.3717 | train_acc: 0.8805 | test_loss: 0.8805 | test_acc: 0.7880
Epoch: 24 | train_loss: 0.3209 | train_acc: 0.8980 | test_loss: 0.6765 | test_acc: 0.8143
Epoch: 25 | train_loss: 0.2840 | train_acc: 0.9064 | test_loss: 0.6287 | test_acc: 0.8378
Epoch: 26 | train_loss: 0.2517 | train_acc: 0.9183 | test_loss: 0.9883 | test_acc: 0.7588
Epoch: 27 | train_loss: 0.2406 | train_acc: 0.9250 | test_loss: 0.8613 | test_acc: 0.7960
Epoch: 28 | train_loss: 0.2554 | train_acc: 0.9181 | test_loss: 1.0396 | test_acc: 0.7723
Epoch: 29 | train_loss: 0.2132 | train_acc: 0.9332 | test_loss: 0.9845 | test_acc: 0.7627
Early stopping triggered at 10 epochs
[INFO] Loading model from: /kaggle/working/results/maxvit_best